# SWRL Rules Creation 

This notebook implements 5 SWRL rules that infer new class memberships based on existing properties:

1. **BudgetFriendlyActivity**: Activities with `budget_free` or `budget_low`
2. **BadWeatherOption**: Activities with `location_indoor` setting
3. **EnglishFriendlyTour**: Tours available in English
4. **WeekendHours**: Operating hours that apply to Saturday or Sunday
5. **OpenOnWeekend**: Physical venues with weekend operating hours

In [21]:
from pathlib import Path
from owlready2 import *
import os

In [22]:
# Define paths
BASE_DIR = Path(os.getcwd()).parent
ONTOLOGY_PATH = BASE_DIR / "ontologies" / "populated" / "german_city_tourism_populated.owl"
OUTPUT_PATH = BASE_DIR / "ontologies" / "populated" / "german_city_tourism_with_rules.owl"

print(f"Loading ontology from: {ONTOLOGY_PATH}")

Loading ontology from: /Users/conocirone/Desktop/german_travel_companion/ontologies/populated/german_city_tourism_populated.owl


In [23]:
# Load the ontology
onto = get_ontology(str(ONTOLOGY_PATH)).load()

In [ ]:
with onto:
    class BudgetFriendlyActivity(onto.Activity):
        """An activity that is free or low-cost, suitable for budget-conscious travelers."""
        pass
    
    class BadWeatherOption(onto.Activity):
        """An indoor activity suitable for rainy or bad weather days."""
        pass
    
    class EnglishFriendlyTour(onto.Tour):
        """A tour that is available in English, accessible for international tourists."""
        pass
    
    class WeekendHours(onto.OperatingHours):
        """Operating hours that apply to weekend days."""
        pass
    
    class OpenOnWeekend(onto.PhysicalVenue):
        """A physical venue that is open on weekends."""
        pass


In [ ]:
budget_free = onto.budget_free
budget_low = onto.budget_low
location_indoor = onto.location_indoor
lang_english = onto.lang_english
day_saturday = onto.day_saturday  
day_sunday = onto.day_sunday     

print("Retrieved instances:")
print(f"  - budget_free: {budget_free}")
print(f"  - budget_low: {budget_low}")
print(f"  - location_indoor: {location_indoor}")
print(f"  - lang_english: {lang_english}")
print(f"  - day_saturday: {day_saturday}") 
print(f"  - day_sunday: {day_sunday}")     

Retrieved instances:
  - budget_free: german_city_tourism_populated.budget_free
  - budget_low: german_city_tourism_populated.budget_low
  - location_indoor: german_city_tourism_populated.location_indoor
  - lang_english: german_city_tourism_populated.lang_english
  - day_saturday: german_city_tourism_populated.day_saturday
  - day_sunday: german_city_tourism_populated.day_sunday


In [26]:
# Define SWRL rules using Owlready2's Imp (implies) class
with onto:
    # Rule 1a: Activity(?a) ^ hasBudget(?a, budget_free) -> BudgetFriendlyActivity(?a)
    rule1a = Imp()
    rule1a.set_as_rule(
        "Activity(?a), hasBudget(?a, budget_free) -> BudgetFriendlyActivity(?a)"
    )
    
    # Rule 1b: Activity(?a) ^ hasBudget(?a, budget_low) -> BudgetFriendlyActivity(?a)
    rule1b = Imp()
    rule1b.set_as_rule(
        "Activity(?a), hasBudget(?a, budget_low) -> BudgetFriendlyActivity(?a)"
    )
    
    # Rule 2: Activity(?a) ^ hasLocationSetting(?a, location_indoor) -> BadWeatherOption(?a)
    rule2 = Imp()
    rule2.set_as_rule(
        "Activity(?a), hasLocationSetting(?a, location_indoor) -> BadWeatherOption(?a)"
    )
    
    # Rule 3: Tour(?t) ^ hasLanguage(?t, lang_english) -> EnglishFriendlyTour(?t)
    rule3 = Imp()
    rule3.set_as_rule(
        "Tour(?t), hasLanguage(?t, lang_english) -> EnglishFriendlyTour(?t)"
    )
    
    # Rule 5a: Saturday hours -> WeekendHours
    rule5a = Imp()
    rule5a.set_as_rule(
        "OperatingHours(?oh), appliesToDay(?oh, day_saturday) -> WeekendHours(?oh)"
    )
    
    # Rule 5b: Sunday hours -> WeekendHours
    rule5b = Imp()
    rule5b.set_as_rule(
        "OperatingHours(?oh), appliesToDay(?oh, day_sunday) -> WeekendHours(?oh)"
    )
    
    # Rule 6: Venue with weekend hours -> OpenOnWeekend
    rule6 = Imp()
    rule6.set_as_rule(
        "PhysicalVenue(?v), hasOperatingHours(?v, ?oh), WeekendHours(?oh) -> OpenOnWeekend(?v)"
    )

In [27]:
sync_reasoner_hermit(infer_property_values=True)
#sync_reasoner_pellet(infer_data_property_values=True, infer_property_values=True)

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /Users/conocirone/Library/Python/3.9/lib/python/site-packages/owlready2/hermit:/Users/conocirone/Library/Python/3.9/lib/python/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/rw/1_0ppc711wv8ynhlpz9d75mm0000gn/T/tmp17sriiao -Y
* Owlready2 * HermiT took 289.90364813804626 seconds
* Owlready * Equivalenting: owl.Thing DUL.Entity
* Owlready * Equivalenting: DUL.Entity owl.Thing
* Owlready * Reparenting german_city_tourism_populated.DayOfWeek: {DUL.Concept, owl.Thing} => {DUL.Concept}
* Owlready * Reparenting german_city_tourism_populated.OperatingHours: {DUL.TimeInterval, owl.Thing} => {DUL.TimeInterval}
* Owlready * Reparenting DUL.parametrizes: {owl.ObjectProperty, DUL.classifies} => {DUL.classifies}
* Owlready * Reparenting DUL.realizes: {owl.ObjectProperty, DUL.associatedWith} => {DUL.associatedWith}
* Owlready * Reparenting DUL.hasRegion: {owl.ObjectProperty, DUL.as

In [29]:
# Save the ontology with the new rules
onto.save(file=str(OUTPUT_PATH), format="rdfxml")
print(f"Ontology with SWRL rules saved to: {OUTPUT_PATH}")

Ontology with SWRL rules saved to: /Users/conocirone/Desktop/german_travel_companion/ontologies/populated/german_city_tourism_with_rules.owl


## Summary of SWRL Rules

| # | Rule Name | SWRL Syntax | Purpose |
|---|-----------|-------------|----------|
| 1a | BudgetFriendlyActivity (Free) | `Activity(?a), hasBudget(?a, budget_free) -> BudgetFriendlyActivity(?a)` | Classify free activities |
| 1b | BudgetFriendlyActivity (Low) | `Activity(?a), hasBudget(?a, budget_low) -> BudgetFriendlyActivity(?a)` | Classify low-cost activities |
| 2 | BadWeatherOption | `Activity(?a), hasLocationSetting(?a, location_indoor) -> BadWeatherOption(?a)` | Weather-based recommendations |
| 3 | EnglishFriendlyTour | `Tour(?t), hasLanguage(?t, lang_english) -> EnglishFriendlyTour(?t)` | International tourist accessibility |
| 4a | WeekendHours (Saturday) | `OperatingHours(?oh), appliesToDay(?oh, day_saturday) -> WeekendHours(?oh)` | Identify Saturday hours |
| 4b | WeekendHours (Sunday) | `OperatingHours(?oh), appliesToDay(?oh, day_sunday) -> WeekendHours(?oh)` | Identify Sunday hours |
| 5 | OpenOnWeekend | `PhysicalVenue(?v), hasOperatingHours(?v, ?oh), WeekendHours(?oh) -> OpenOnWeekend(?v)` | Find weekend venues |


### Use Cases

- **Budget Planning**: Query `BudgetFriendlyActivity` to find affordable options
- **Weather Adaptation**: Query `BadWeatherOption` when weather is bad
- **Tourist Accessibility**: Query `EnglishFriendlyTour` for non-German speakers
- **Weekend Planning**: Query `OpenOnWeekend` to find venues open on weekends
